In [1]:
import os 
import csv
import pandas  as pd
import numpy as np
from datasets import load_dataset
import requests 
from bs4 import BeautifulSoup
from newspaper import Article, ArticleException
from huggingface_hub import hf_hub_download
import pandas as pd

In [3]:
def getArticleContent(url): 
    
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # need to figure out how to abstractly find the 'body' of an article 
    body = soup.find('div', class_='main')
    

In [4]:
def getData(csv_file): 
    
    rows = 0
    with open(csv_file) as my_file: 
        
        reader = csv.reader(my_file)
        next(reader) # skip the header 
        
        for row in reader: 
            # each row contains four links 
            # left article, center article, right article, and original page
            # some links are missing
            getArticleContent(row[1])
            break
        


In [5]:
getData('Part1.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Part1.csv'

In [5]:
REPO_ID = "valurank/PoliticalBias"

df1 = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename='Part1.csv', repo_type="dataset")
)
df1 = pd.DataFrame(df1)
df2 = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename='Part2.csv', repo_type="dataset")
)
df2 = pd.DataFrame(df2)
df = df1._append(df2)
df = df.drop(columns='Main URL')
df.to_csv('urls.csv')


In [41]:
for index, row in df.iterrows():
    
    # Iterate over items in the row
    for column, value in row.items():
        if value != np.nan:
            try:
                url = str(value)
                article = Article(url)
                article.download()
                article.parse()
            except ArticleException:
                print('***FAILED TO DOWNLOAD***', url)

In [9]:
REPO_ID = "mediabiasgroup/BABE"

df1 = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename='BABE.csv', repo_type="dataset")
)
df1 = pd.DataFrame(df1)

df1 = df1.dropna()

df1.to_csv('urls_BABE.csv')


EntryNotFoundError: 404 Client Error. (Request ID: Root=1-66126288-20f417b448f67a3a4f75f2d1;07d28797-7294-4158-baac-c696940fb028)

Entry Not Found for url: https://huggingface.co/datasets/mediabiasgroup/BABE/resolve/main/BABE.csv.

In [2]:
def relable_class(row):
    if row['type_class'] == 'left':
        return 0
    elif row['type_class'] == 'center':
        return 1
    elif row['type_class'] == 'right':
        return 2
    else:
        return row['type_class']  # Return the original value if not one of the specified types

In [3]:
# Scrape data using URLs

df1 = pd.read_csv('urls_BABE.csv')
df_scraped = pd.DataFrame(columns=['url', 'content', 'type_class'])
print(len(df1))
x, hit, miss = 0, 0, 0

for index, row in df1.iterrows():
    if(x == len(df1)-1):
        break
    # Iterate over items in the row
    url = row['news_link']
    url = str(url)
    type_class = row['type']
    if url != np.nan:
        try:
            article = Article(url)
            article.download()
            article.parse()
            df_scraped = df_scraped._append({'url': url, 'content': article.text, 'type_class': type_class}, ignore_index=True)
            hit += 1
            #print('Succesfully grabbed article #' + str(x))
            print(f"\rProgress: {hit} articles pulled... {round(x/len(df1)*100,1)}%", end="")
        except ArticleException:
            miss += 1
            #print('***FAILED TO DOWNLOAD***')
    x +=1
print('\nSuccesfully scraped ' + str(hit) + ' articles. Missed ' + str(miss))
#print(df_scraped.head(20))

# Output scraped data csv
df_scraped['type_class'] = df_scraped.apply(relable_class, axis=1)
df_scraped.to_csv('scraped_data/BABE_scraped.csv')
type_counts = df_scraped['type_class'].value_counts() # eval number of each type_class
print(type_counts)

2644
Progress: 1630 articles pulled... 91.1%
Succesfully scraped 1630 articles. Missed 1013
type_class
0    860
2    627
1    143
Name: count, dtype: int64
